In [20]:
import os
import re
import time
import string

import xlrd
from collections import Counter

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
# import pre_cor
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings
warnings.filterwarnings('ignore')

# 保险机构名单

In [2]:
sel_org = pd.read_excel('data/insurance_org_list_20180824.xlsx')
print(sel_org.shape)
sel_org.head()

(178, 1)


,name
0,仁和人寿保险股份有限公司
1,中国人民财产保险股份有限公司
2,中国人寿保险股份有限公司
3,中国平安人寿保险股份有限公司
4,中国太平洋人寿保险股份有限公司


In [3]:
circ_keyword = pd.read_excel('data/wise_class_tree_node_keyword.xlsx')
print(circ_keyword.shape)
circ_keyword.head()

(375, 9)


,id,name,query_nature,query_and,query_or,query_not,classify_id,node_id,flag
0,36,保监会,0,NaN,保险监督管理委员会 保监会 CIRC 银保监会 银行保险监督管理委员会,NaN,7,37,0
1,37,陈文辉,0,陈文辉,保险监督管理委员会 保监 CIRC 银保监,NaN,7,38,0
2,38,黄洪,0,黄洪,保险监督管理委员会 保监 CIRC 银保监,NaN,7,39,0
3,39,梁涛,0,梁涛,保险监督管理委员会 保监 CIRC 银保监,NaN,7,40,0
4,40,曹宇,0,曹宇,保险监督管理委员会 保监 CIRC 银保监,NaN,7,41,0


In [4]:
org_names = pd.merge(sel_org, circ_keyword, on = 'name', how = 'left')
print(org_names.shape)
org_names.head()

(178, 9)


,name,id,query_nature,query_and,query_or,query_not,classify_id,node_id,flag
0,仁和人寿保险股份有限公司,336.0,0.0,NaN,招商仁和 仁和人寿 仁和人寿保险 仁和寿险,NaN,9.0,337.0,0.0
1,中国人民财产保险股份有限公司,115.0,0.0,NaN,人保财险 人民财产保险 人保产险 人保车险,NaN,9.0,118.0,0.0
2,中国人寿保险股份有限公司,196.0,0.0,NaN,中国人寿保险股份 中国人寿股份 国寿股份,NaN,9.0,199.0,0.0
3,中国平安人寿保险股份有限公司,199.0,0.0,NaN,平安人寿 平安人寿保险 平安寿险 平安寿,NaN,9.0,202.0,0.0
4,中国太平洋人寿保险股份有限公司,195.0,0.0,NaN,太保寿险 太保人寿 太平洋人寿保险 太平洋人寿 太平洋寿险,NaN,9.0,198.0,0.0


## 机构字典

In [17]:
org_name_list = org_names['name'].tolist()

for value in org_names['query_or']:
    value = str(value)
    if value not in ['nan', '']:
        name_list = value.split(' ')
        print(len(name_list), name_list)
        org_name_list += name_list
    else :
        print('****  ', value)

org_name_list = [v for v in org_name_list if v != '']        
print(len(org_name_list))
org_name_list

4 ['招商仁和', '仁和人寿', '仁和人寿保险', '仁和寿险']
4 ['人保财险', '人民财产保险', '人保产险', '人保车险']
3 ['中国人寿保险股份', '中国人寿股份', '国寿股份']
4 ['平安人寿', '平安人寿保险', '平安寿险', '平安寿']
5 ['太保寿险', '太保人寿', '太平洋人寿保险', '太平洋人寿', '太平洋寿险']
6 ['平安财产保险', '平安财险', '', '平安财保', '平安产险', '平安车险']
4 ['人保寿险', '人保人寿', '人民人寿保险', '中国人民人寿']
3 ['太平人寿', '太平人寿保险', '太平寿险']
4 ['华夏人寿', '华夏人寿保险', '华夏寿险', '华夏保险']
3 ['泰康人寿', '泰康人寿保险', '泰康寿险']
4 ['新华人寿', '新华人寿保险', '新华寿险', '新华保险']
5 ['富德生命', '富德生命人寿保险', '富德生命人寿', '富德生命寿险', '富德生命保险']
4 ['太平财险', '太平财产保险', '太平财保', '太平产险']
7 ['太保产险', '太保财险', '太保财产保险', '太平洋财产保险', '太平洋财险', '太平洋产险', '太平洋财保']
4 ['国华人寿', '国华人寿保险', '国华保险', '国华寿险']
3 ['前海人寿', '前海人寿保险', '前海寿险']
4 ['中邮人寿', '中邮人寿保险', '中邮保险', '中邮寿险']
4 ['国泰产险', '国泰财产保险', '国泰财险', '国泰财保']
7 ['中国人寿财产保险', '中国人寿财保', '中国人寿财险', '中国人寿产险', '国寿财险', '国寿产险', '国寿财保']
3 ['建信人寿', '建信人寿保险', '建信寿险']
3 ['天安人寿', '天安人寿保险', '天安寿险']
3 ['百年人寿', '百年人寿保险', '百年寿险']
4 ['君康人寿', '君康人寿保险', '君康寿险', '君康保险']
3 ['阳光人寿', '阳光人寿保险', '阳光寿险']
5 ['大地保险', '大地财产保险', '大地财险', '大地财保', '大地产险']
5 ['工银安盛', '工银安盛人寿保险', '工

['仁和人寿保险股份有限公司',
 '中国人民财产保险股份有限公司',
 '中国人寿保险股份有限公司',
 '中国平安人寿保险股份有限公司',
 '中国太平洋人寿保险股份有限公司',
 '中国平安财产保险股份有限公司',
 '中国人民人寿保险股份有限公司',
 '太平人寿保险有限公司',
 '华夏人寿保险股份有限公司',
 '泰康人寿保险股份有限公司',
 '新华人寿保险股份有限公司',
 '富德生命人寿保险股份有限公司',
 '太平财产保险有限公司',
 '中国太平洋财产保险股份有限公司',
 '国华人寿保险股份有限公司',
 '前海人寿保险股份有限公司',
 '中邮人寿保险股份有限公司',
 '国泰财产保险有限责任公司',
 '中国人寿财产保险股份有限公司',
 '建信人寿保险股份有限公司',
 '天安人寿保险股份有限公司',
 '百年人寿保险股份有限公司',
 '君康人寿保险股份有限公司',
 '阳光人寿保险股份有限公司',
 '中国大地财产保险股份有限公司',
 '工银安盛人寿保险有限公司',
 '恒大人寿保险有限公司',
 '农银人寿保险股份有限公司',
 '中华联合财产保险股份有限公司',
 '阳光财产保险股份有限公司',
 '安邦人寿保险股份有限公司',
 '中国人民健康保险股份有限公司',
 '利安人寿保险股份有限公司',
 '友邦保险有限公司',
 '平安养老保险股份有限公司',
 '合众人寿保险股份有限公司',
 '光大永明人寿保险有限公司',
 '中信保诚人寿保险有限公司',
 '招商信诺人寿保险有限公司',
 '天安财产保险股份有限公司',
 '信泰人寿保险股份有限公司',
 '幸福人寿保险股份有限公司',
 '英大泰和财产保险股份有限公司',
 '交银康联人寿保险有限公司',
 '民生人寿保险股份有限公司',
 '中意人寿保险有限公司',
 '中国出口信用保险公司',
 '英大泰和人寿保险股份有限公司',
 '珠江人寿保险股份有限公司',
 '华安财产保险股份有限公司',
 '泰康养老保险股份有限公司',
 '中美联泰大都会人寿保险有限公司',
 '上海人寿保险股份有限公司',
 '渤海人寿保险股份有限公司',
 '中英人寿保险有限公司',
 '国元农业保险股份有限公司',
 '鼎和财产保险股份有限公司',
 '中宏人寿保险有限公司',
 

# 读入数据

In [12]:
file_list = os.listdir('data/circ_Q2_data')
circ_Q2_data = pd.DataFrame()

for file_name in file_list:
    file_data = pd.read_excel('data/circ_Q2_data/%s'%file_name)
    print('file_name: ', file_name, '  shape: ', file_data.shape)
    circ_Q2_data = pd.concat([circ_Q2_data, file_data], axis = 0)
    
print(circ_Q2_data.shape)
circ_Q2_data.head()

file_name:  circ_Q2_data_2018-04-01.xlsx   shape:  (14061, 4)
file_name:  circ_Q2_data_2018-04-02.xlsx   shape:  (25951, 4)
file_name:  circ_Q2_data_2018-04-03.xlsx   shape:  (24280, 4)
file_name:  circ_Q2_data_2018-04-04.xlsx   shape:  (25006, 4)
file_name:  circ_Q2_data_2018-04-05.xlsx   shape:  (14264, 4)
(103562, 4)


,id,publishtime,title,content
0,7518044,2018-04-01 00:00:00,【新规】明天起，这些新规将影响你的生活,来 源：人民日报、北京晨报综合 版权归原作者所有，如有侵权请与我们联系 ...
1,7518108,2018-04-01 00:10:00,QFII的投资布局逐渐显露 新进24家上市公司(名单),作为外资的重要代表力量，QFII的持仓变化一向受到市场各方关注。随着2017年上市公司年...
2,7518133,2018-04-01 00:06:54,全球飞机租赁公司机队价值最新排名前20位，你都认识了吗？,原标题：全球飞机租赁公司机队价值最新排名前20位，你都认识了吗？全球飞机租赁公司机队价值最新...
3,7518154,2018-04-01 00:10:07,新规落地 券商资管、信托通道变阵,3月28日，《关于规范金融机构资产管理业务的指导意见》（以下简称《资管新规》）获得中央全面深...
4,7518209,2018-04-01 00:10:07,独角兽IPO刚刚定了！国务院重磅宣布：阿里腾讯京东能回A股！,刚刚，国务院办公厅转发证监会关于开展创新企业境内发行股票或存托凭证试点若干意见的通知，证监会...


In [13]:
circ_Q2_data.info(memory_usage = 'deep') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103562 entries, 0 to 14263
Data columns (total 4 columns):
id             103562 non-null int64
publishtime    103562 non-null datetime64[ns]
title          103431 non-null object
content        103394 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 406.5 MB


# 分词计算

In [ ]:
file_list = os.listdir('data/circ_Q2_data')

for index, file_name in enumerate(file_list):
    file_name_count = file_name.replace('data_', 'data_count_')
    day = file_name_count[-15:-5]
    print(index, file_name, file_name_count)
    if not os.path.exists('data/circ_Q2_data_count/%s'%file_name_count):
        file_data = pd.read_excel('data/circ_Q2_data/%s'%file_name)
        print('file_name: ', file_name, '  shape: ', file_data.shape)
        file_data['title_content'] = file_data['title'].astype(str) + '。' + file_data['content'].astype(str)
        
        data_pre = pre_cor.handle_contents(file_data.loc[:, 'title_content'].tolist())
        count_word = Counter(" ".join(data_pre).split())
        
        org_name_dict = {}
        for org in org_name_list:
            if org in count_word:
                org_name_dict[org] = count_word[org]
#                 print(org, count_word[org])
            else :
                org_name_dict[org] = 0        
        
        org_count_res = pd.DataFrame.from_dict(org_name_dict, orient='index')
        org_count_res = org_count_res.reset_index()
        org_count_res.columns = ['name', 'count_%s'%day]
        
        org_count_res.to_excel('data/circ_Q2_data_count/%s'%file_name_count, index = False)

0 circ_Q2_data_2018-04-01.xlsx circ_Q2_data_count_2018-04-01.xlsx
1 circ_Q2_data_2018-04-02.xlsx circ_Q2_data_count_2018-04-02.xlsx
2 circ_Q2_data_2018-04-03.xlsx circ_Q2_data_count_2018-04-03.xlsx
3 circ_Q2_data_2018-04-04.xlsx circ_Q2_data_count_2018-04-04.xlsx
file_name:  circ_Q2_data_2018-04-04.xlsx   shape:  (25006, 4)


# 合并分词计算结果

In [ ]:
file_list = os.listdir('data/circ_Q2_data_count')

org_count = pd.DataFrame()
for index, file_name_count in enumerate(file_list):
    name_count = pd.read_excel('data/circ_Q2_data_count/%s'%file_name_count)
    org_count = pd.merge(org_count, name_count, on = 'name')
    
print(org_count.shape)
org_count.head()